In [0]:
#モデルの構築
text = ' '
files = os.listdir('song')
for f in files[:]:
  with open('song/' + f) as f:
    text += f.read().lower()

text = text.split()
dic = sorted(list(set(text)))
index_w=[]
word_i  =[]

index_w = dict((i,word) for i, word in enumerate(dic))
word_i   = dict((word,i) for i, word in enumerate(dic))

x = np.zeros((len(dic), len(dic)), dtype='int64')
for i in range(len(dic)):
  x[i][i] = 1

y = np.zeros((len(dic), len(dic)), dtype='int64')
for i in range(0,len(text)-2):
  y[word_i[text[i]]][word_i[text[i+1]]] += 1
  y[word_i[text[i]]][word_i[text[i+2]]] += 1
  
for i in range(2,len(text),-1):
  y[word_i[text[i]]][word_i[text[i-1]]] += 1
  y[word_i[text[i]]][word_i[text[i-2]]] += 1
  
model = Sequential()
model.add(Dense(64, input_dim=len(dic)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(len(dic), activation = 'softmax'))

model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=['categorical_accuracy'])
model.fit(x,y,batch_size=32,epochs=10)


In [0]:
#近くにある単語を出力
for i in range(10):
  pred_x = np.zeros((1,len(dic)), dtype='int64')
  pred_x[0,word_i[text[i]]] = 1
  print(text[i])
  p = model.predict(pred_x,verbose=0)[0]
  p = np.asarray(p).astype('float64')
  for j in range(5):
    print(index_w[np.argsort(p)[::-1][j]], np.sort(p)[::-1][j])
    
  print()

In [0]:
#cos類似度を計算
def cos_similar(v1,v2):
  return np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

for start in range(10):
  cosvalue = np.empty(len(dic))
  for i in range(len(dic)):
    cosvalue[i] =  cos_similar(model.get_weights()[0][start], model.get_weights()[0][i])
  print(dic[start])
  for j in range(10):
    print(index_w[np.argsort(cosvalue)[::-1][j]], np.sort(cosvalue)[::-1][j])